---
# MNIST em uma CNN usando Keras
---

From: https://github.com/fchollet/keras-resources

## Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs (there is still a lot of margin for parameter tuning).

16 seconds per epoch on a GRID K520 GPU.

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import os
os.environ['THEANO_FLAGS'] = "device=gpu1,floatX=float32"    

import theano

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


Não está executando na GPU! Precisa ficar ligado porque ele só mostra o warning da primeira vez. Da primeira vez que ocorreu eu tentei mudar o número da GPU e o warning sumiu. Acreditei que havia funcionado, mas não é o caso.

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu1 is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)

## Tentando solucionar o problema

Página com os pacotes do CUDA e algumas instruções para instalação: https://developer.nvidia.com/cuda-downloads

Tentei instalar o CUDA diretamente na máquina do AWS. Segui os seguintes passos:

Tentativa com pacote ``.rpm`` tanto local quanto pela rede:

``wget <link_download_cuda_8.0>
sudo yum clean all
sudo yum install cuda``

Não funciona, ele reclama de várias bibliotecas faltando e não instala.

Tentativa com ``runfile`` local:

``sudo yum install -y gcc kernel-devel
wget <link_download_cuda_8.0>
sudo sh cuda_8.0.44_linux.run``

E após isto aceitar o passo a passo da instalação.

Parece que instalou corretamente, entretanto não pude instalar os exemplos, por falta de espaço e o erro ao tentar executar o nvidia-docker, para testá-lo (``nvidia-docker run --rm nvidia/cuda nvidia-smi``), mudou, anteriormente ele dizia que não encontrar o nvidia-modprobe e que ele provavelmente não estava instalado.
O erro atual é o seguinte: 

``docker: Error response from daemon: create nvidia_driver_367.48: create nvidia_driver_367.48: Error looking up volume plugin nvidia-docker: plugin not found.``


## Problema ainda não solucionado!

Pode estar relacionado à versão do Docker no repositório para instalação não ser a mais recente, no que é recomendado desinstalar este docker e instalar o novo através do repositório oficial, seguindo os seguintes passos:

``sudo yum install -y yum-utils
sudo yum-config-manager \
     --add-repo \
     https://docs.docker.com/engine/installation/linux/repo_files/centos/docker.rep
sudo yum-config-manager
sudo yum makecache fast
sudo yum -y install docker-engine``

Entretanto ele dá erro neste último passo e não instala devido à pacotes não encontrados. Dentre outros ele reclama do systemd, que pelo que li não existe (?) para o RHEL (Red Hat Enterprise Linux) 6, mas para o 7 acho que sim.
De qualquer forma não sei qual a versão a Amazon usa e se este problema está relacionado, pois parece que a imagem deles é uma mistura do RHEL e do CentOS.

Talvez seja mais recomendado usar um Ubuntu padrão ao invés do Linux da Amazon.

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using Theano backend.


In [3]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [4]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Criação do modelo da rede neural convolucional

In [9]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

## Treinando o modelo com o conjunto de treino separado

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 116s - loss: 0.3898 - acc: 0.8816 - val_loss: 0.0991 - val_acc: 0.9699
Epoch 2/12
60000/60000 [==============================] - 116s - loss: 0.1412 - acc: 0.9581 - val_loss: 0.0639 - val_acc: 0.9799
Epoch 3/12
60000/60000 [==============================] - 121s - loss: 0.1041 - acc: 0.9693 - val_loss: 0.0512 - val_acc: 0.9834
Epoch 4/12
49408/60000 [=======================>......] - ETA: 19s - loss: 0.0920 - acc: 0.9727

## Testando a acurácia do modelo com o conjunto de teste separado

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])